In [ ]:
# prompt: connect drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil

src_folder = 'drive/MyDrive/ML/coil-20-proc'
dst_folder = 'coil20-organized'

os.makedirs(dst_folder, exist_ok=True)

for fname in os.listdir(src_folder):
    if fname.endswith('.png'):
        label = fname.split('__')[0]  # e.g., obj1
        label_path = os.path.join(dst_folder, label)
        os.makedirs(label_path, exist_ok=True)
        shutil.copy(os.path.join(src_folder, fname), os.path.join(label_path, fname))

print("Images organized successfully into class folders.")


Images organized successfully into class folders.


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # ResNet expects 3 channels
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dataset = datasets.ImageFolder('/content/coil20-organized', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Get class names
class_names = dataset.classes
print(f"Classes: {class_names}")


Classes: ['obj1', 'obj10', 'obj11', 'obj12', 'obj13', 'obj14', 'obj15', 'obj16', 'obj17', 'obj18', 'obj19', 'obj2', 'obj20', 'obj3', 'obj4', 'obj5', 'obj6', 'obj7', 'obj8', 'obj9']


In [ ]:
# prompt: train resnet 50 on this, using train test split as well

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import random_split

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create data loaders for training and validation sets
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load the pre-trained ResNet-50 model
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

# Modify the output layer for the number of classes in your dataset
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10  # Adjust as needed

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataloader.dataset)
    epoch_accuracy = correct_predictions / total_samples

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.4f}')

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    running_loss_val = 0.0
    correct_predictions_val = 0
    total_samples_val = 0

    with torch.no_grad():
        for inputs, labels in val_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss_val += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_samples_val += labels.size(0)
            correct_predictions_val += (predicted == labels).sum().item()

    epoch_loss_val = running_loss_val / len(val_dataloader.dataset)
    epoch_accuracy_val = correct_predictions_val / total_samples_val
    print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {epoch_loss_val:.4f}, Validation Accuracy: {epoch_accuracy_val:.4f}')

print("Training finished.")


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 130MB/s]


Epoch 1/10, Train Loss: 0.5169, Train Accuracy: 0.8872
Epoch 1/10, Validation Loss: 0.3366, Validation Accuracy: 0.8958
Epoch 2/10, Train Loss: 0.0421, Train Accuracy: 0.9922
Epoch 2/10, Validation Loss: 3.4960, Validation Accuracy: 0.7882
Epoch 3/10, Train Loss: 0.0450, Train Accuracy: 0.9870
Epoch 3/10, Validation Loss: 0.2161, Validation Accuracy: 0.9410
Epoch 4/10, Train Loss: 0.0148, Train Accuracy: 0.9965
Epoch 4/10, Validation Loss: 0.4519, Validation Accuracy: 0.8993
Epoch 5/10, Train Loss: 0.0705, Train Accuracy: 0.9800
Epoch 5/10, Validation Loss: 1.6444, Validation Accuracy: 0.6285
Epoch 6/10, Train Loss: 0.0650, Train Accuracy: 0.9861
Epoch 6/10, Validation Loss: 0.0134, Validation Accuracy: 0.9965
Epoch 7/10, Train Loss: 0.0681, Train Accuracy: 0.9878
Epoch 7/10, Validation Loss: 0.0410, Validation Accuracy: 0.9757
Epoch 8/10, Train Loss: 0.0537, Train Accuracy: 0.9852
Epoch 8/10, Validation Loss: 0.0744, Validation Accuracy: 0.9792
Epoch 9/10, Train Loss: 0.0099, Train Ac